# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

## Import Packages

In [1]:
# Importing some useful packages
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from helper_functions import *
import pickle

#Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib qt


## 1 - Compute the camera calibration using chessboard images

In [2]:
# define the name of the output directory to be created
output_path = "output_images/camera_calibration"

# Create test_images_output Directory if doesn't exist
if not os.path.exists(output_path):
    os.mkdir(output_path)

In [3]:
# To avoid running the calibration step everytime, in case the camera coefficients already exist, just load it
if os.path.exists('camera_coeff.pkl'):
    # Getting back the values:
    with open('camera_coeff.pkl', 'rb') as f:  
        mtx, dist = pickle.load(f)
    
else:  
    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(9,6,0)
    objp = np.zeros((9*6,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:

        # Read the image
        img = cv2.imread(fname)

        # Convert the image to gray scale
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard 54 (9*6) corners 
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, append object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            #cv2.imshow('img',img)
            #cv2.waitKey(500)

    #cv2.destroyAllWindows()

    # Perform the camera calibration
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None) 

    # Store the camera calibration coefficients for future use
    with open('camera_coeff.pkl', 'wb') as f:  
        pickle.dump([mtx, dist], f)
        
    # Perform distortion correction on chessboard images to verify the process is doing well

    # Step through the list and undistort each image
    for fname in images:

        # Read the image
        image = mpimg.imread(fname)

        # Undistort the image an display it 
        undist = cv2.undistort(image, mtx, dist, None, mtx)

        # Save images on output_path Directory
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
        f.tight_layout()

        ax1.imshow(image)
        ax1.set_title('Original Image', fontsize=30)

        ax2.imshow(undist)
        ax2.set_title('Undistorted Image', fontsize=30)
        plt.subplots_adjust(left=0.1, right=0.9, top=1, bottom=0, wspace = 0.1)


        plt.savefig(os.path.join(output_path, "undist_" + os.path.basename(fname)))
        plt.close()

## 2 - Build a Lane finding Pipeline
The pipeline will be composed by the following steps:

1 - Read and apply a distortion correction to raw images

2 - Use color transforms, gradients, etc., to create a thresholded binary image

3 - Apply a perspective transform to rectify binary image ("birds-eye view")

4 - Detect lane pixels and fit to find the lane boundary

5 - Determine the curvature of the lane and vehicle position with respect to center 

6 - Warp the detected lane boundaries back onto the original image

7 - Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position

In [4]:
# define the name of the directory to be created
output_path = "output_images/test_images"

# Create test_images_output Directory if doesn't exist
if not os.path.exists(output_path):
    os.mkdir(output_path)

In [5]:
# Get the transform matrix using 4 source and destination points calculated manually looking to 
# the straight line images

src = np.float32([[195, 720],[1125, 720],[578, 460],[705, 460]])
dst = np.float32([[350, 720],[950, 720],[350,0],[950,0]])

# Get the transform matrix M using the src and dst points
M = cv2.getPerspectiveTransform(src, dst)

# Get the invert transform matrix M using the src and dst points
M_inv = cv2.getPerspectiveTransform(dst, src)
        

In [6]:
def process_image(original_image, fname, mtx, dist, M, M_inv):
    
    ## 1 - Apply a distortion correction to the image ##
    undist = undistort_image(original_image, mtx, dist)
    save_undistorted_images(output_path, fname, original_image, undist)

    ## 2 - Use color transforms, gradients, etc., to create a thresholded binary image ##
    ksize = 3 # Sobel kernel size 

    # Apply each of the gradient thresholding functions
    gradx = abs_sobel_thresh(undist, orient='x', sobel_kernel=ksize, thresh=(30, 200))
    grady = abs_sobel_thresh(undist, orient='y', sobel_kernel=ksize, thresh=(50, 200))
    mag_binary = mag_thresh(undist, sobel_kernel=ksize, mag_thresh=(30, 200))
    dir_binary = dir_threshold(undist, sobel_kernel=ksize, thresh=(0.7, 1.3))

    # Apply each of the color thresholding functions for HLS color space
    hls_colors_binary = hls_select(undist, s_thresh=(170, 240), l_thresh=(200, 255)) # l - 200

    # Apply each of the color thresholding functions for HSV color space
    hsv_colors_binary = hsv_select(undist, s_thresh=(130,255), v_thresh=(240, 255), vs_thresh=(200, 255)) # v - 200
        
    # Combine all of the thresholding binaries
    binary_image = np.zeros_like(mag_binary)
    binary_image[(hsv_colors_binary == 1) | (hls_colors_binary == 1) | ((mag_binary == 1) & (dir_binary == 1)) ] = 1
    save_binary_images(output_path, fname, undist, binary_image)
    
    # 3 - Apply a perspective transform to rectify binary image ("birds-eye view") ##

    # Warp the image to a top-down view
    img_size = (undist.shape[1], undist.shape[0])
    binary_warped = cv2.warpPerspective(binary_image, M, img_size, flags=cv2.INTER_LINEAR)
    
    warped = cv2.warpPerspective(undist, M, img_size, flags=cv2.INTER_LINEAR)
    save_warped_images(output_path, fname, binary_warped, warped)

    ## 4 - Detect lane pixels and fit to find the lane boundary ##
    
    # Create a sliding window and find out which activated pixels fall into the window
    out_img, left_fit, right_fit, left_fitx, right_fitx, ploty = fit_polynomial(binary_warped)

    ## 5 - Determine the curvature of the lane and vehicle position with respect to center ##
    
    radius_curvature, left_radius_curvature , right_radius_curvature = measure_curvature_real(out_img.shape[0], left_fit, right_fit)
    
    rel_vehicle_position = measure_rel_vehicle_position(out_img.shape, left_fit, right_fit)

    ## 6 - Warp the detected lane boundaries back onto the original image ##

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, M_inv, img_size) 

    ## 7 - Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position ##
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    result = cv2.addWeighted(original_image, 1, newwarp, 0.3, 0)

    save_lane_lines_image(output_path, fname, out_img)

    print("Radius curvature = ", radius_curvature, 'm')
    print("Left Radius curvature = ", left_radius_curvature, 'm', "Right Radius curvature = ", right_radius_curvature, 'm', )
    print("Relative vehicle position with respect to the line lane center = ",rel_vehicle_position, 'm')

    save_pipeline_image(output_path, fname, result, radius_curvature, rel_vehicle_position )

In [7]:
%matplotlib inline

# Make a list of test images
images = glob.glob('test_images/*.jpg')
images = []

# Step through the list, read the image and apply the lane finding pipeline
for fname in images:

    print("-----------------------", fname, "-----------------------")
    #fname = "test_images/test4.jpg"
    # Read the image
    img = mpimg.imread(fname)
    img_size = img.shape

    process_image(img, fname, mtx, dist, M, M_inv)

    print('\n')

## 3 -  Test the pipeline on Videos

In [8]:
from frame import *
img_size = (720,1280)

In [9]:
#if True:
if False:
    white_output = 'project_video_out.mp4'
    clip1 = VideoFileClip("project_video.mp4")
    white_clip = clip1.fl_image(Frame(mtx, dist, M, M_inv, img_size, 2)).subclip(0,5)
    %time white_clip.write_videofile(white_output, audio=False)


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

NameError: name 'white_output' is not defined

## 4 - Test the pipeline on Challenge Videos

In [ ]:
# Get the transform matrix using 4 source and destination points calculated looking to the straight line image
src = np.float32([[250, 720],[1050, 720],[605, 480],[715, 480]])
dst = np.float32([[350, 720],[950, 720],[350,0],[950,0]])

# Get the transform matrix M using the src and dst points
M_1 = cv2.getPerspectiveTransform(src, dst)

# Get the invert transform matrix M using the src and dst points
M_inv_1 = cv2.getPerspectiveTransform(dst, src)

fname = "challenge_video.jpg"

# Read the image
clip1 = VideoFileClip("challenge_video.mp4")
img = clip1.get_frame(52 / clip1.fps) # get frame by index
img_size = img.shape

process_image(img, fname, mtx, dist, M_1, M_inv_1)

print('\n')

In [11]:
white_output = 'challenge_video_out.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(Frame(mtx, dist, M_1, M_inv_1, img_size,2)).subclip(0,5) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   0%|          | 0/150 [00:00<?, ?it/s, now=None]

Moviepy - Building video challenge_video_out.mp4.
Moviepy - Writing video challenge_video_out.mp4



t:   7%|▋         | 11/150 [00:02<00:35,  3.95it/s, now=None]

error 1 955.0 2007.0


t:   8%|▊         | 12/150 [00:02<00:35,  3.90it/s, now=None]

...

error 5 776.565662570802 970.4791389261075 375.9568867899249


t: 100%|██████████| 150/150 [00:36<00:00,  4.88it/s, now=None]

error 1 254.0 3302.0
error 5 776.7865430741241 978.3556732903522 372.26164616860643


error 1 271.0 1563.0
error 5 771.8636017417365 980.1538173206061 369.8385541560398


Moviepy - Done !
Moviepy - video ready challenge_video_out.mp4
CPU times: user 1min 2s, sys: 10.1 s, total: 1min 12s
Wall time: 38.6 s


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [10]:
def process_image_1(original_image, fname, mtx, dist, M, M_inv):
    
    ## 1 - Apply a distortion correction to the image ##
    undist = undistort_image(original_image, mtx, dist)
    save_undistorted_images(output_path, fname, original_image, undist)

    ## 2 - Use color transforms, gradients, etc., to create a thresholded binary image ##
    ksize = 3 # Sobel kernel size 

    # Apply each of the gradient thresholding functions
    gradx = abs_sobel_thresh(undist, orient='x', sobel_kernel=ksize, thresh=(30, 200))
    grady = abs_sobel_thresh(undist, orient='y', sobel_kernel=ksize, thresh=(50, 200))
    mag_binary = mag_thresh(undist, sobel_kernel=ksize, mag_thresh=(30, 200))
    dir_binary = dir_threshold(undist, sobel_kernel=ksize, thresh=(0.7, 1.3))

    # Apply each of the color thresholding functions for HLS color space
    hls_colors_binary = hls_select(undist, s_thresh=(170, 240), l_thresh=(200, 255)) # l - 200

    # Apply each of the color thresholding functions for HSV color space
    hsv_colors_binary = hsv_select(undist, s_thresh=(130,255), v_thresh=(240, 255), vs_thresh=(200, 255)) # v - 200
        
    # Combine all of the thresholding binaries
    binary_image = np.zeros_like(mag_binary)
    binary_image[(hsv_colors_binary == 1) | (hls_colors_binary == 1) | ((mag_binary == 1) & (dir_binary == 1)) ] = 1
    save_binary_images(output_path, fname, undist, binary_image)
    
    # 3 - Apply a perspective transform to rectify binary image ("birds-eye view") ##

    # Warp the image to a top-down view
    img_size = (undist.shape[1], undist.shape[0])
    binary_warped = cv2.warpPerspective(binary_image, M, img_size, flags=cv2.INTER_LINEAR)
    
    warped = cv2.warpPerspective(undist, M, img_size, flags=cv2.INTER_LINEAR)
    save_warped_images(output_path, fname, binary_warped, warped)

    ## 4 - Detect lane pixels and fit to find the lane boundary ##
    
    # Create a sliding window and find out which activated pixels fall into the window
    out_img, left_fit, right_fit, left_fitx, right_fitx, ploty = fit_polynomial(binary_warped)

    ## 5 - Determine the curvature of the lane and vehicle position with respect to center ##
    
    radius_curvature, left_radius_curvature , right_radius_curvature = measure_curvature_real(out_img.shape[0], left_fit, right_fit)

    rel_vehicle_position = measure_rel_vehicle_position(out_img.shape, left_fit, right_fit)

    ## 6 - Warp the detected lane boundaries back onto the original image ##

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, M_inv, img_size) 

    ## 7 - Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position ##
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    result = cv2.addWeighted(original_image, 1, newwarp, 0.3, 0)

    save_lane_lines_image(output_path, fname, out_img)

    print("Radius curvature = ", radius_curvature, 'm')
    print("Left Radius curvature = ", left_radius_curvature, 'm', "Right Radius curvature = ", right_radius_curvature, 'm', )
    print("Relative vehicle position with respect to the line lane center = ",rel_vehicle_position, 'm')

    save_pipeline_image(output_path, fname, result, radius_curvature, rel_vehicle_position )

In [11]:
# Get the transform matrix using 4 source and destination points calculated looking to the straight line image
src = np.float32([[250, 720],[1050, 720],[605, 480],[715, 480]])
dst = np.float32([[350, 720],[950, 720],[350,0],[950,0]])

# Get the transform matrix M using the src and dst points
M_1 = cv2.getPerspectiveTransform(src, dst)

# Get the invert transform matrix M using the src and dst points
M_inv_1 = cv2.getPerspectiveTransform(dst, src)

fname = "challenge_video.jpg"

# Read the image
clip1 = VideoFileClip("harder_challenge_video.mp4")
img = clip1.get_frame(20 / clip1.fps) # get frame by index
img_size = img.shape

process_image_1(img, fname, mtx, dist, M, M_inv)

print('\n')

336
897
327
927
317
900
302
842
287
814
265
798
224
747
167
699
90
675
teste
Radius curvature =  205.0 m
Left Radius curvature =  111.0 m Right Radius curvature =  300.0 m
Relative vehicle position with respect to the line lane center =  -0.06 m




In [15]:
white_output = 'harder_challenge_video_out.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(Frame(mtx, dist, M, M_inv, img_size, 2)).subclip(0,5) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

error 1 803.0 313.0


t:   0%|          | 0/125 [00:00<?, ?it/s, now=None]

error 1 719.0 311.0
Moviepy - Building video harder_challenge_video_out.mp4.
Moviepy - Writing video harder_challenge_video_out.mp4



t:   2%|▏         | 2/125 [00:00<00:17,  7.23it/s, now=None]

error 1 682.0 308.0


...

error 5 755.3379896831011 946.8812357051474 330.7433301328393


t: 100%|██████████| 125/125 [00:36<00:00,  3.57it/s, now=None]

error 5 754.7615411881521 962.1122259044985 331.22506586740633


error 5 753.6478289068507 978.26168049962 333.9460781061499


Moviepy - Done !
Moviepy - video ready harder_challenge_video_out.mp4
CPU times: user 1min 9s, sys: 12 s, total: 1min 21s
Wall time: 39.6 s


In [16]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))